# MSDM 5058 Information Science Assignment 3

In [1]:
import numpy as np
from itertools import product
from gamepandas import GameFrame, ZeroSumGameFrame

In this assignment, I realized it's tedious that just calculate and write with hands, thus I decided to develop a python package called `gamepandas` to finish the task in this assignment.

If you want to see the source code of `gamepandas`, you can visit the repo on github: [https://github.com/Algebra-FUN/GamePandas](https://github.com/Algebra-FUN/GamePandas).

If you want to install it, just input installation command since I had already uploaded it to `pypi`.

```shell
pip install gamepandas
```

## (1) Dominated Moves
Each one of two bars charges its own price for a beer, either $2, $4, or $5. The
cost of obtaining and serving the beer can be neglected. It is expected that 6000 beers per month are drunk in a bar by tourists, who choose one of the two bars randomly, and 4000 beers per month are drunk by natives who go to the bar with the lowest price, and split evenly in case both bars offer the same price. What prices would the bars select?

### (a) Construct the payoff table for the two bars

In [2]:
def generateQ1Table(prices=(2,4,5),tourists=6,natives=4):
    N = len(prices)
    table = [[None]*N for _ in range(N)]
    for (i,p),(j,q) in product(enumerate(prices),repeat=2):
        P = tourists/2*p
        Q = tourists/2*q
        if q < p:
            Q += natives*q
        elif p < q:
            P += natives*p
        else:
            P += natives/2*p
            Q += natives/2*q
        table[i][j] = (P,Q)
    return GameFrame.create(table,prices,prices)

In [3]:
Q1Table = generateQ1Table()
Q1Table

,2,4,5
2,"(10.0, 10.0)","(14.0, 12.0)","(14.0, 15.0)"
4,"(12.0, 14.0)","(20.0, 20.0)","(28.0, 15.0)"
5,"(15.0, 14.0)","(15.0, 28.0)","(25.0, 25.0)"


### (b) Use elimination process to find the dominant strategies for the two bars. 
What is the price that each bar will choose, and what is the payoff?

In [4]:
Q1Table.eliminate()

### Step 1: Eliminate row[2]

,2,4,5
4,"(12.0, 14.0)","(20.0, 20.0)","(28.0, 15.0)"
5,"(15.0, 14.0)","(15.0, 28.0)","(25.0, 25.0)"


### Step 2: Eliminate column[2]

,4,5
4,"(20.0, 20.0)","(28.0, 15.0)"
5,"(15.0, 28.0)","(25.0, 25.0)"


### Step 3: Eliminate row[5]

,4,5
4,"(20.0, 20.0)","(28.0, 15.0)"


### Step 4: Eliminate column[5]

,4
4,"(20.0, 20.0)"


,4
4,"(20.0, 20.0)"


Finally, the dominat strategy is ($4,$4), the payoff is ($20k,$20k)

## (2) Domination of Strategies
Let us look at a zero-sum game between two players with the following payoff matrix

$$
\begin{bmatrix}
3&5&3\\
4&-3&2\\
3&2&3
\end{bmatrix}
$$

### (a) Use elimination procedures to find the saddle point of this game. What is its value?

In [5]:
Q2Matrix = np.array([[3,5,3],[4,-3,2],[3,2,3]])
Q2Table = ZeroSumGameFrame.create(Q2Matrix,['A','B','C'],['A','B','C'])
Q2Table

,A,B,C
A,3,5,3
B,4,-3,2
C,3,2,3


In [6]:
Q2Table.eliminate()

### Step 1: Eliminate row[C]

,A,B,C
A,3,5,3
B,4,-3,2


### Step 2: Eliminate column[A]

,B,C
A,5,3
B,-3,2


### Step 3: Eliminate row[B]

,B,C
A,5,3


### Step 4: Eliminate column[B]

,C
A,3


,C
A,3


Finally, the dominat strategy is (A,C), the payoff is (3,-3).

### (b) Are there other optimal solutions for this game? 
If so, find the solutions and their values (Hint: Look for mixed strategies)

#### The row player optimization problem:

Minimize $x'$

subject to 

$$
A^{\top} x' \geq 1
$$

and $x'\geq 1$,

where $A$ is the payoff matrix.

The strategies weight of row player is $w_x = x' / \sum_i x'_i$

#### The colunm player optimization problem:

Maximize $y'$

subject to 

$$
A y' \leq 1
$$

and $x'\geq 1$,

where $A$ is the payoff matrix.

The strategies weight of column player is $w_y = y' / \sum_j y'_j$

In [7]:
Q2RowPStrategy = Q2Table.row_player_mixed_strategy
Q2RowPStrategy

A    1.0
B    0.0
C    0.0
dtype: float64

In [8]:
Q2Table.row_player_payoff(Q2RowPStrategy)

3.0

The row player optimal strategy is (1,0,0). The payoff is 3.

In [9]:
Q2ColPStrategy = Q2Table.col_player_mixed_strategy
Q2ColPStrategy

A    0.5
B    0.0
C    0.5
dtype: float64

In [10]:
Q2Table.col_player_payoff(Q2ColPStrategy)

3.0

The column player optimal strategy is (0.5,0,0.5). The payoff is 3.

### (c) 

#### What is the reason for having optimal solutions with both pure and mixed strategies? 

Because this game has a saddle point, then there must be a optimal solution for pure strategy scheme, this solution also must be the solution for mixed strategy scheme. If there exists another optimal solution for mixed strategy scheme, then there are optimal solutions.

#### What is the most general solution of this game?

The mixed strategy scheme is more general, it's also suitable for the game without saddle point.

## (3) Mixed Strategies
The administration of a multinational company and the union of workers are
preparing to sit down at the bargaining table to work out the details of a new contract for the workers. Each side has developed certain proposals for the contents of the new contract. Let us call union proposals “Proposal 1”, “Proposal 2” and “Proposal 3”, and administration’s proposals “Contract A”, “Contract B” and “Contract C”. Both parties

are aware of the financial aspects of each proposal–contract combination. The payoff matrix is as follows

|Proposal\Contract|A|B|C|
|:--:|:--:|:--:|:--:|
|1|9.5|12|7|
|2|7|8.5|6.5|
|3|6|9|10|

### (a) Is there an equilibrium point?

It's easy to see, there is no pure-strategy Nash equilibirum point. Then people should consider mixed strategies.

### (b) Find the mixed strategies for the union and the administration. 

Let $M$ denotes the payoff matrix of this game. 

$E_{union}$ denotes the expected value of union and $E_{admin}$ denotes the expected value of admin.

Since this game is a zero-sum game, then $E_{admin}=-E_{union}$.

Denotes $x$ is the strategy vector of union and $y$ is the strategy vector of admin, should statisify $\sum x=1,\sum y=1$

Then,

$$
E:=E_{union}=x^\top M y.
$$

Denotes $(x^*,y^*)$ is the mixed-strategy Nash equilibirum point.

$x^*$ should be the solution for this optimization problem:

$$
\min_y E\\
s.t.\quad y^\top \vec{1} = 1 
$$

Using Lagrange Multiplier method, transform this optimization problem to solve this equation:

$$
\begin{cases}
\frac{\partial E}{\partial y}+\lambda\frac{\partial}{\partial y}y^\top \vec{1}&=0\\
y^\top \vec{1} &= 1 
\end{cases}
$$

After simplification,

$$
\begin{cases}
x^\top M+\lambda\vec{1}^\top&=0\\
y^\top \vec{1} &= 1 
\end{cases}
$$

After simplification,

$$
\begin{bmatrix}M^\top&\vec{1}\end{bmatrix}
\begin{bmatrix}x\\\lambda\end{bmatrix}
=0
$$

Meanwhile, $y^*$ should be the solution for this optimization problem:

$$
\max_x E\\
s.t.\quad x^\top \vec{1} = 1 
$$

Similarly,

$$
\begin{bmatrix}M&\vec{1}\end{bmatrix}
\begin{bmatrix}y\\\lambda\end{bmatrix}
=0
$$



In [11]:
Q3Matrix = np.array([[9.5,12,7],[7,8.5,6.5],[6,9,10]])
Q3Table = ZeroSumGameFrame(Q3Matrix,(1,2,3),['A','B','C'])
Q3Table

,A,B,C
1,9.5,12.0,7.0
2,7.0,8.5,6.5
3,6.0,9.0,10.0


In [12]:
ElimQ3Table = Q3Table.eliminate()
ElimQ3Table

### Step 1: Eliminate row[2]

,A,B,C
1,9.5,12.0,7.0
3,6.0,9.0,10.0


### Step 2: Eliminate column[B]

,A,C
1,9.5,7.0
3,6.0,10.0


,A,C
1,9.5,7.0
3,6.0,10.0


In [13]:
UnionStrategy = ElimQ3Table.row_player_nash_equilibrium_mixed_strategies
UnionStrategy

,0
1,0.615385
3,0.384615


In [14]:
AdminStrategy = ElimQ3Table.col_player_nash_equilibrium_mixed_strategies
AdminStrategy

,0
A,0.461538
C,0.538462


Result:

The strategy of union is (0.6154,0,0.3846)

The strategy of admin is (0.4615,0,0.5385)

### (c) What is the value of the game?

In [15]:
ElimQ3Table.game_value(UnionStrategy,AdminStrategy)

,0
0,8.153846


## (4) Bertrand game with differentiated products
If two firms have the same constant marginal cost, they earn zero profits in the
                    Bertrand equilibrium. This depends crucially on the feature that the goods involved are perfect substitutes. If products are differentiated instead, then the Bertrand equilibrium can lead to positive profits. The products are differentiated when consumers consider them only imperfect substitutes. While a consumer may be unwilling to buy the product of one producer, she will have the incentive to do so if the price of her favourite product becomes too high. To model this, we allow the demand for each good to depend not only on its own price but also on the price of the other good.

Assume the demand $q_i$ are described by the following functions:
$$
\begin{cases}
q_1=180-p_1-(p_1-\bar{p})\\
q_2=180-p_2-(p_2-\bar{p})
\end{cases}
$$
where $\bar{p}$ is the average price taken over the prices of the two firms ($=\frac{p_1+p_2}2$).

Assume that each firm has an average (and marginal) cost c = 20. Suppose the firms can only choose between the three prices: 92, 82, 72.

### (a) Compute the profits of the firms under the 9 different price combinations that are
possible in the model. Use you answer to construct the 3 x 3 payoff matrix for the
normal form game where the payoffs are given by the profits of the firms

In [16]:
def demand_func(price,average_price,const=180):
    return const - 2*price + average_price 

def generateQ4Table(prices=(72,82,92),cost=20,const=180):
    N = len(prices)
    table = [[None]*N for _ in range(N)]
    for (i,p1),(j,p2) in product(enumerate(prices),repeat=2):
        ps = (p1,p2)
        bar_p = sum(ps)/2
        qs = [demand_func(p,bar_p,const) for p in ps]
        table[i][j] = tuple((p-cost)*q for p,q in zip(ps,qs))
    return GameFrame.create(table,prices,prices)

In [17]:
Q4Table = generateQ4Table()
Q4Table

,72,82,92
72,"(5616.0, 5616.0)","(5876.0, 5766.0)","(6136.0, 5616.0)"
82,"(5766.0, 5876.0)","(6076.0, 6076.0)","(6386.0, 5976.0)"
92,"(5616.0, 6136.0)","(5976.0, 6386.0)","(6336.0, 6336.0)"


### (b) Find the (Bertrand-)Nash equilibrium of this game. What are the profits for the firms at this equilibrium?

In [18]:
list(Q4Table.pure_nash_equilibriums)

[(82, 82)]

In [19]:
Q4Table.at[82,82]

(6076.0, 6076.0)